# Import modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
import pytz
import datetime 
import astral
import astropy.time as aptime
import os

file paths & save paths

In [2]:
data_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/datatables/'

In [93]:
save_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Preliminary/'

In [3]:
txt_file1 = pd.read_csv(data_path + 'IFS_RawContrast_Gemini_env_20180530_all_seeing.csv')
txt_file1_copy = copy.copy(txt_file1)
raw_contrast_data= pd.DataFrame(txt_file1_copy)
print len(raw_contrast_data)

raw_contrast_data['dts'] = aptime.Time(raw_contrast_data['MJDOBS'], format='mjd').datetime 
raw_contrast_data['dts'] = raw_contrast_data['dts'].dt.tz_localize(pytz.UTC)

24155


In [ ]:
txt_file2 = pd.read_csv(data_path + 'sp_psd_summary20181126.csv',index_col=0)
txt_file2_copy = copy.copy(txt_file2)
sp_psd_data = pd.DataFrame(txt_file2_copy)

In [97]:
sp_psd_data['dts'] = pd.to_datetime(sp_psd_data.loc[:,'filename'].str.split('_').str.get(2) + '_' + sp_psd_data.loc[:,'filename'].str.split('_').str.get(3),
              utc=True, format = '%Y.%m.%d_%H.%M.%S')

sp_psd_data['dts'] = sp_psd_data['dts'].dt.tz_localize(pytz.UTC)
sp_psd_data['whenstr'] = sp_psd_data['whenstr'].astype('str')

Merge PSD data with IFS data

In [6]:
def filter_data(data, imag = 10., tau = 1., see = 2., all_cond = True):
    "This selects all rows of input dataframe that satisfy conditions set by the user"
    
    date1 = datetime.date(year = 2014, month = 1, day =1)
    
    #  AO system requirements
    cond1 = data['dts'] > date1 
    cond2 = data['COADDS'] == 1
    cond3 = (data['OBSMODE'] == 'H_coron')|(data['OBSMODE'] == 'Spec')
    cond4 = data['AOFRAMES'] == 1000

    #  Good seeing conditions
    cond5 = data['IMAG'] < imag #Bright Stars
    cond6 = data['MASSTAU'] > tau  # slow moving turbulence [ms]
    cond7 = data['DIMMSEE'] < see  # smaller scale turbulence ["]
    
    if all_cond:
        ind = np.where(cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7)[0]  
        print len(ind)
    else:
        ind = np.where(cond1 & cond2 & cond3 & cond4 & cond5)[0]  
        print len(ind)

    filtered_data = data.iloc[ind]
    #filtered_data = filtered_data.dropna(subset = ['CONTR040','cal_wfe','M1_avg','TAMBIENT','Outside_OE_temperature'])
    
    new_ind = np.arange(len(filtered_data))
    filtered_data = filtered_data.set_index(new_ind)
        
    return filtered_data

In [101]:
r = filter_data(raw_contrast_data,all_cond=False)

17041


In [102]:
sp_psd_raw_IFS_data =  pd.merge_asof(raw_contrast_data,sp_psd_data[['slope','dts']].sort_values('dts'), on='dts',tolerance=pd.Timedelta('1min'))

In [103]:
print sum(np.isfinite(sp_psd_raw_IFS_data['slope']))

30


In [104]:
%matplotlib notebook

plt.figure()
plt.scatter(sp_psd_raw_IFS_data['M1_avg']-sp_psd_raw_IFS_data['TAMBIENT'],sp_psd_raw_IFS_data['slope']
            ,c=sp_psd_raw_IFS_data['night_number'],alpha=.5)
plt.ylabel('Spatial PSD Slope',fontsize=15)
plt.xlabel('Primary - Outside air [C]',fontsize=15)
plt.savefig(save_path+'sp_psd_slope_vs_delT_night_20181119.pdf')
#plt.plot(sp_psd_raw_IFS_data['slope'],'.',alpha=.5)

<IPython.core.display.Javascript object>

In [81]:
len(pd.unique(sp_psd_raw_IFS_data[['dts','slope','M1_avg','TAMBIENT','night_number']].dropna()['night_number']))

32

In [46]:
sp_psd_raw_IFS_data[['dts','slope','M1_avg','TAMBIENT','night_number']].dropna()

dts     slope  M1_avg  TAMBIENT  \
105   2014-11-09 03:57:24.413760+00:00 -3.661848  12.810      14.2   
414   2014-11-11 00:39:13.155839+00:00 -3.375269  13.570      10.8   
456   2014-11-11 02:45:06.382080+00:00 -3.138347  13.145      10.1   
465   2014-11-11 02:56:42.636480+00:00 -2.940396  13.045      10.0   
1829  2015-01-29 04:25:10.197120+00:00 -3.281668  15.330      13.1   
1969  2015-01-30 02:29:33.478080+00:00 -3.200935  15.915      15.7   
2907  2015-04-06 06:50:06.184320+00:00 -3.567005  13.400      13.6   
4377  2015-07-02 07:27:33.094080+00:00 -3.054863   7.600       4.0   
4386  2015-07-02 07:38:47.938560+00:00 -2.984668   7.580       4.1   
4755  2015-07-03 07:27:41.690880+00:00 -3.078428   6.505       4.7   
4756  2015-07-03 07:28:56.651520+00:00 -3.045423   6.505       4.7   
4758  2015-07-03 07:31:27.851520+00:00 -3.029079   6.515       4.6   
4992  2015-07-04 03:28:37.701121+00:00 -3.356738   7.180       8.3   
5881  2015-08-29 00:26:48.050880+00:00 -3.385162  12.525      13.8   
5898  2015-08-29 00:48:37.131841+00:00 -3.367535  12.540      13.7   
5972  2015-08-29 02:48:12.332160+00:00 -3.252255  12.645      13.9   
5978  2015-08-29 02:55:55.237440+00:00 -3.419065  12.685      13.9   
5993  2015-08-29 03:26:07.131841+00:00 -3.348176  12.795      14.0   
5994  2015-08-29 03:27:24.148800+00:00 -3.319695  12.795      14.0   
6007  2015-08-29 03:44:04.133760+00:00 -3.321114  12.745      13.9   
6009  2015-08-29 03:46:40.578240+00:00 -3.386352  12.770      13.9   
6111  2015-08-30 03:05:45.288960+00:00 -3.439822  14.560      17.1   
6133  2015-08-30 03:45:41.472000+00:00 -3.406112  14.625      17.2   
6144  2015-08-30 03:59:46.550400+00:00 -3.495384  14.700      16.9   
6197  2015-08-30 06:48:07.678080+00:00 -3.459588  14.740      16.5   
6420  2015-08-31 03:13:19.390080+00:00 -2.968402  15.175      16.4   
6441  2015-08-31 03:40:17.446080+00:00 -3.310517  15.220      16.5   
6672  2015-09-01 02:54:35.317440+00:00 -3.484744  16.425      17.3   
6680  2015-09-01 03:06:07.165440+00:00 -3.460932  16.425      17.6   
6693  2015-09-01 03:22:44.126400+00:00 -3.460818  16.445      17.7   
...                                ...       ...     ...       ...   
13518 2016-09-21 00:12:15.678720+00:00 -3.111652  16.130      14.9   
13529 2016-09-21 00:26:05.170560+00:00 -3.155698  16.130      14.9   
13545 2016-09-21 00:57:00.835199+00:00 -3.220952  16.130      14.8   
13549 2016-09-21 01:02:02.163840+00:00 -3.299474  16.095      14.8   
13581 2016-09-21 02:09:38.712960+00:00 -3.163176  16.085      14.9   
13584 2016-09-21 02:13:24.026880+00:00 -3.216941  16.085      14.9   
13592 2016-09-21 02:26:01.063679+00:00 -3.271140  16.100      14.9   
13597 2016-09-21 02:33:32.132160+00:00 -3.144679  16.085      14.8   
13599 2016-09-21 02:36:02.926080+00:00 -3.111267  16.085      14.9   
13600 2016-09-21 02:37:18.802560+00:00 -3.143058  16.085      14.9   
13663 2016-09-21 05:37:05.704320+00:00 -3.153129  16.005      14.6   
13683 2016-09-21 06:14:29.840640+00:00 -3.264684  16.010      14.5   
13697 2016-09-21 06:32:06.555840+00:00 -3.151065  16.035      14.5   
13864 2016-09-22 00:36:12.890880+00:00 -3.229009  15.905      13.4   
13867 2016-09-22 00:40:40.696320+00:00 -3.162381  15.880      13.7   
13880 2016-09-22 00:57:20.430720+00:00 -3.231677  15.875      13.8   
13886 2016-09-22 01:04:51.853440+00:00 -3.284335  15.855      13.8   
13902 2016-09-22 01:51:33.373440+00:00 -3.225872  15.770      13.7   
13905 2016-09-22 01:56:07.373761+00:00 -3.352969  15.855      13.6   
13925 2016-09-22 02:21:18.146880+00:00 -3.182221  15.740      13.3   
13975 2016-09-22 04:59:05.807040+00:00 -3.400451  15.675      13.6   
13997 2016-09-22 05:28:04.641601+00:00 -3.297137  15.635      13.5   
14007 2016-09-22 06:01:30.849600+00:00 -3.017134  15.580      13.4   
14146 2016-10-15 00:43:22.031040+00:00 -2.889983  10.815       7.8   
14150 2016-10-15 00:48:21.605760+00:00 -3.004310  10.790       8.1   
14622 2016-11-18 03:35:10.432320+00:00 -3.259